# 자체 시퀀스 생성

- python만의 인덱싱/슬라이싱 기능은 **__getitem__**라는 매직메소드 덕분에 동작한다.
- myobject[key]와 같은 형태를 사용할 때 호출되는 메소드로 key에 해당하는 대괄호 안의 값을 파라미터로 전달한다.


- 클래스가 시퀀스임을 선언하기 위해 collections.abc 모듈의 Sequence 인터페이스를 구현해야한다.

In [1]:
from collections.abc import Sequence

class Items:
    def __init__(self, *values):
        self._values = list(values)
    
    def __len__(self):
        return len(self._values)
    
    def __getitem__(self, item):
        return self._values.__getitem__(item)

In [13]:
print(Items(9, 8, 7, 6, 5).__len__())
print(Items(9, 8, 7, 6, 5).__getitem__(1))

5
8


# 컨텍스트 관리자(context manager)

- 컨텍스트 관리자는 파이썬이 제공하는 유용한 기능
- 패턴에 잘 대응되기 때문에 유용함
- 사전 조건과 사후 조건이 있는 일부 코드를 실행하는 경우에 사용할 수 있음


- 리소스 관리와 관련해 파일을 열면 파일 디스크립터 누수를 막기 위해 작업이 끝나면 적절히 닫히길 기대함
- 일반적으로 할당된 모든 리소스를 해제해야하지만 예외가 발생한 경우 혹은 오류를 처리해야 하는 경우 가능한 모든 조합과 실행 경로를 처리하여 디버깅하는 것이 어렵다는 점을 감안해 일반적으로 finally 블록에 정리 코드를 넣는다.

In [21]:
def process_file(file):
    print(f"{file.name} is opend")
    return None

filename = "./data/test.txt"
fd = open(filename)
try:
    process_file(fd)
finally:
    print(f"{filename} is closed")
    fd.close()

./data/test.txt is opend
./data/test.txt is closed


위와 똑같은 기능을 다음과 같이 파이썬스러운 방법으로 구현할 수 있다.

In [22]:
with open(filename) as fd:
    process_file(fd)

./data/test.txt is opend


- with 문(pep-343)은 컨텍스트 관리자로 진입하게 한다.
- 이 경우 open 함수는 컨텍스트 관리자 프로토콜을 구현한다.
- 예외가 발생한 경우에도 블록이 완료되면 파일을 자동으로 닫는다.


- 컨텍스트 관리자는 \_\_enter\_\_, \_\_exit\_\_ 두개의 매직 메소드로 구성된다.
- 첫번째 줄에서 with문은 \_\_enter\_\_ 메소드를 호출하고 이후 무엇을 반환하든 as 이후 지정된 변수에 할당된다.(fd)
- 이후 마지막 문장이 끝나면 컨텍스트가 종료되며 파이썬이 호출한 원래 컨텍스트 관리자 객체의 \_\_exit\_\_ 메소드를 호출한다.

예를 들어 스크립트를 통해 DB 백업을 진행하는 경우를 생각해보자.

주의사항은 다음과 같다.

- 백업은 오프라인 상태에서 해야한다.
- 백업이 끝나면 백업 프로세스가 성공적으로 진행되었는지 여부에 관계 없이 프로세스를 다시 시작해야 한다.

ref : https://docs.python.org/3/reference/datamodel.html#object.__exit__

In [24]:
def stop_database():
    run("systemctl stop postgresql.service")
    
def start_database():
    run("systemctl start postgresql.service")
    
class DBHandler:
    def __enter__(self):
        stop_database()
        return self
    
    def __exit__(self, exc_type, ex_value, ex_traceback):
        start_database()
    
    def db_backup():
        run("pg_dump database")
        
    # main 메소드에서 유지보수 작업과 상관없이 백업을 실행한다. 또한 백업에 오류가 있어도 __exit__를 호출한다.
    # __exit__가 True를 반환하는 경우 잠재적으로 발생한 예외를 호출자에게 전달하지 않고 멈춘다는 것을 의미한다.
    # 이 경우 
    def main():
        with DBHandler():
            db_backupup()

# 프로퍼티, 속성(Attribute)과 객체 메소드의 다른 타입들

- public/private/protected와 같은 접근 제어자를 가지는 다른 언어들과 다르게 파이썬은 객체의 모든 속성과 함수를 public으로 가져간다.
- 즉 호출자가 객체의 속성을 호출하지 못하도록 할 방법이 없다.
- 파이썬에는 변수명 지정과 관련한 몇 가지 네이밍 컨벤션이 있다.
- **밑줄로 시작하는 속성은 private 속성을 의미하며 외부에서 호출되지 않기를 기대한다는 의미이다.**

## 단일 밑줄

In [36]:
from dataclasses import dataclass

@dataclass
class Connector:
    def __init__(self, source: str):
        self.source = source
        self._timeout = 60
        
conn = Connector("postgresql://localhost")
conn

Connector()

In [37]:
conn.source

'postgresql://localhost'

In [38]:
conn._timeout

60

In [39]:
conn.__dict__

{'source': 'postgresql://localhost', '_timeout': 60}

위 Connector 객체는 source 파라미터를 통해 생성되며 source, timeout 속성을 가진다.

여기서 timeout의 경우는 private이지만 위 예시처럼 호출자가 접근 가능하다.

위 코드에서 \_timeout 속성은 connector 자체에서만 사용되고 호출자는 이 속성에 접근하지 말아야 한다.

따라서 네이밍 룰이 제대로 지켜졌다면 호출자는 밑줄을 확인하고 외부에서 호출하지 않으며 안전하게 리팩토링을 진행할 수 있을 것이다.

timeout 속성을 이중 밑줄로 정의한 경우

## 이중 밑줄

In [46]:
from dataclasses import dataclass

@dataclass
class Connector:
    def __init__(self, source: str):
        self.source = source
        self.__timeout = 60
        
    def connect(self):
        print(f"connecting with {self.__timeout}s")
        return
    
conn = Connector("postgresql://localhost")
conn.connect()
conn.__timeout

connecting with 60s


AttributeError: 'Connector' object has no attribute '__timeout'

밑줄 두개를 사용하면 파이썬은 다른 이름을 만드는 name mangling 이 발생한다.

- \_\<class_name>\_\_\<attribute_name> 형태로 변환됨

In [48]:
conn._Connector__timeout

60

파이썬에서 이중 밑줄을 사용하는 것은 여러 번 확장되는 클래스의 메소드를 이름 충돌 없이 오버라디으 하기 위해 만들어졌다. 따라서 이중 밑줄을 사용하는 경우 위와 같은 상황을 주의해야 한다.

## 프로퍼티

- 일반적인 객체 지향 설계에서는 도메인 엔티티를 추상화하는 객체를 만든다.
- 이러한 객체는 어떠한 동작이나 데이터를 캡슐화할 수 있다.
- 데이터 정확성이 객체 생성 가능 여부를 결정해 데이터가 특정 값을 가질때만 객체를 생성할 수 있도록 설계할 수 있다.
- 일반적으로 setter 작업에 사용되는 유효성 검사 메소드를 파이썬에서는 프로퍼티를 통해 setter/getter 메소드를 더 간결하게 캡슐화할 수 있다.

In [72]:
class Coordinate:
    def __init__(self, lat: float, long: float) -> None:
        self._latitude = self._longitude = None
        self.latitude = lat
        self.longitude = long
        
    @property
    def latitude(self) -> float:
        return self._latitude

    @latitude.setter
    def latitude(self, lat_value: float) -> None:
        if lat_value not in range(-90, 90+1):
            raise ValueError(f"유효하지 않은 위도 값 : {lat_value}")
        self._latitude = lat_value
        
    @property
    def longitude(self) -> float:
        return self._longitude
    
    @longitude.setter
    def longitude(self, long_value: float) -> None:
        if long_value not in range(-180, 180+1):
            raise ValueError(f"유효하지 않은 경도 값 : {long_value}")
        self._longitude = long_value
        

In [77]:
coor = Coordinate(50, 150)
coor.__dict__

{'_latitude': 50, '_longitude': 150}

In [82]:
# 클래스의 속성값을 다음과 같이 변경 가능하다
# 유효성 검사 동일

coor.latitude = 25
coor.__dict__

{'_latitude': 25, '_longitude': 150}

In [88]:
class Person:
    def __init__(self, name, age):
        self._name = self._age = None
        self.name = name
        self.age = age
        
    @property
    def age(self):
        return self._age
    
    @property
    def name(self):
        return self._name
    
    @age.setter
    def age(self, age_val):
        if age_val > 100:
            raise ValueError("ff")
        self._age = age_val
        
    @name.setter
    def name(self, name_val):
        self._name = name_val

In [91]:
Person("K", 110).__dict__

ValueError: ff